# Otimização Pandas

In [21]:
# importando libs necessarias
import pandas as pd
import numpy as np

In [22]:
# Função que mede o uso de memória de cada coluna e do df
def memory(df):
    memStats = df.memory_usage()
    print("Consumo de cada coluna em bytes:")
    print(memStats)
    print("Consumo do DataFrame em bytes:%d bytes"%(memStats.sum()))
    print("Consumo do DataFrame em megabytes(MB): %2.2f MB"%(memStats/1024 ** 2).sum())
    return memStats.sum()

In [23]:
# importando aquivo csv com o pandas
df = pd.read_csv('SP_poluicao_dados.csv')

In [24]:
# excluindo colunas duplicatadas
df.drop(['Unnamed: 0','ID'], axis=1, inplace=True)

In [25]:
# verificando as 5 primeiras linhas do DataFrame
df.head()

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo
0,2015-01-01,01:00,Americana - Vila Santa Maria,SP01,MP10,65.0,ug/m3,automatica
1,2015-01-01,02:00,Americana - Vila Santa Maria,SP01,MP10,98.0,ug/m3,automatica
2,2015-01-01,03:00,Americana - Vila Santa Maria,SP01,MP10,79.0,ug/m3,automatica
3,2015-01-01,04:00,Americana - Vila Santa Maria,SP01,MP10,53.0,ug/m3,automatica
4,2015-01-01,05:00,Americana - Vila Santa Maria,SP01,MP10,35.0,ug/m3,automatica


In [26]:
#Verificando o consumo de memória do DF sem alterações
m1 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     87844120
Codigo      87844120
Poluente    87844120
Valor       87844120
Unidade     87844120
Tipo        87844120
dtype: int64
Consumo do DataFrame em bytes:702753088 bytes
Consumo do DataFrame em megabytes(MB): 670.20 MB


Pode-se verificar que o DF está usando cerca de 670 MB de memoria, vamos tentar diminuir utilizando tecnicas simples.

## Regras de Otimização
https://wiki.c2.com/?RulesOfOptimizationClub
 1. You do not optimize.
 2. You do not optimize, without measuring first.
 3. When the performance is not bound by the code, but by external factors, the optimization is over.
 4. Only optimize code that already has full unit test coverage.
 5. One factor at a time.
 6. No unresolved bugs, no schedule pressure.
 7. Testing will go on as long as it has to.
 8. If this is your first night at Optimization Club, you have to write a test case.

In [27]:
#Verificando o tipo de cada coluna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980515 entries, 0 to 10980514
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Data      object 
 1   Hora      object 
 2   Estacao   object 
 3   Codigo    object 
 4   Poluente  object 
 5   Valor     float64
 6   Unidade   object 
 7   Tipo      object 
dtypes: float64(1), object(7)
memory usage: 670.2+ MB


In [28]:
print(f'Numero de unicos {df["Estacao"].nunique()}, equivale a {(df["Estacao"].nunique() / len(df)) * 100 :.7f}% dos valores')
# Regra de ouro, caso os valores únicos sejam menos de 50% da quantidade de observações, transforma-se em category

Numero de unicos 87, equivale a 0.0007923% dos valores


In [29]:
#Transformando o tipo para 'Category'
df['Estacao'] = df['Estacao'].astype('category')

In [30]:
#Verificando a quatidade de memória economizada
m2 = memory(df)
print(f'{(m1 - m2) / (1024 ** 2):.2f} MB')
print(f'{(m1-m2)/m1*100:.2f}%')

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      87844120
Poluente    87844120
Valor       87844120
Unidade     87844120
Tipo        87844120
dtype: int64
Consumo do DataFrame em bytes:625892283 bytes
Consumo do DataFrame em megabytes(MB): 596.90 MB
73.30 MB
10.94%


### Apenas com uma alteração, diminuimos o uso de memória em mais **10%!!!**

In [31]:
# Vamos repetir para as demais colunas que podem ser categóricas
for coluna in ['Codigo','Poluente','Unidade','Tipo']:
    print(coluna)
    print(f'Numero de unicos {df[coluna].nunique()}, {(df[coluna].nunique() / len(df)) * 100 :.7f}%')
    print('------------')

Codigo
Numero de unicos 87, 0.0007923%
------------
Poluente
Numero de unicos 9, 0.0000820%
------------
Unidade
Numero de unicos 1, 0.0000091%
------------
Tipo
Numero de unicos 2, 0.0000182%
------------


#### Nenhuma coluna tem mais de 50% de valores únicos, portanto, podemos aplicar a mesma técnica e economizar ainda mais

In [32]:
## Aplicando a mesma transformação para as demais colunas e verificando a quantidade de memória
for coluna in ['Codigo','Poluente','Unidade','Tipo']:
    df[coluna] = df[coluna].astype('category')

m3 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      10983315
Poluente    10980887
Valor       87844120
Unidade     10980631
Tipo        10980639
dtype: int64
Consumo do DataFrame em bytes:318441275 bytes
Consumo do DataFrame em megabytes(MB): 303.69 MB


In [33]:
# Precisamos tomar um cuidado extra com o tipo float, transformando de float64 para float32, diminuímos a precisão dos valores (nesse caso, as casas após a virgula)
df['Valor'] = df['Valor'].astype('float32')
m4 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      10983315
Poluente    10980887
Valor       43922060
Unidade     10980631
Tipo        10980639
dtype: int64
Consumo do DataFrame em bytes:274519215 bytes
Consumo do DataFrame em megabytes(MB): 261.80 MB


In [34]:
# Percebemos que os campos de data e hora também podem ser convertidos para categorias
df['Data'].describe()
df['Hora'].describe()

count     10980515
unique          24
top          17:00
freq        469259
Name: Hora, dtype: object

In [35]:
# Transformando e medindo
df['Data'] = df['Data'].astype('category')
df['Hora'] = df['Hora'].astype('category')
m5 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        22047574
Hora        10981263
Estacao     10983315
Codigo      10983315
Poluente    10980887
Valor       43922060
Unidade     10980631
Tipo        10980639
dtype: int64
Consumo do DataFrame em bytes:131859812 bytes
Consumo do DataFrame em megabytes(MB): 125.75 MB


In [36]:
# Comparações finais
print(f'Inicial: {m1 / (1024 ** 2):.2f} MB')
print(f'Final: {m5 / (1024 ** 2):.2f} MB')
print(f'Diferença: {(m1 - m5) / (1024 ** 2):.2f} MB')
print(f'Diminuição de {(m1-m5)/m1*100:.2f}%')
print(f'M6 equivale a {m5/m1*100:.2f}% do M1')

Inicial: 670.20 MB
Final: 125.75 MB
Diferença: 544.45 MB
Diminuição de 81.24%
M6 enquivale a 18.76% do M1


### Em poucas linhas de código e em apenas alguns minutos, pudemos diminuir a quantidade de memória utilizada em **5X!!!**